In [1]:
# tylko nowe cechy
# liczba nowych cech to połowa oryginalnych cech
# sqrt(n) zaokrąglony w góre
# klasyfikatory - zafiksowac sensowne ustawienia
# wagi sumowane do 1
# lecim po wszystkich zbiorach czemu nie
# k-fold cross validation
from multi_imbalance.datasets import load_datasets
import experiment2 as ex
from sklearn.model_selection import StratifiedKFold
from collections import Counter, defaultdict
from multi_imbalance.utils.metrics import gmean_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

/home/dhorna/anaconda3/envs/mgr/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
datasets = load_datasets()

In [3]:
config = ex.config_calculation_strategy2(datasets)

In [4]:
config

{'1czysty-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 12,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 '2delikatne-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 12,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 '3mocniej-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 12,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 '4delikatne-bezover-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 12,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 'balance-scale': {'nn_config': {'units_1st_layer': 4,
   'units_2nd_layer': 24,
   'units_3rd_layer': 12,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 'cleveland': {'nn_config': {'units_1st_layer': 13,
   'units_2nd_layer': 39,
   'units_3rd_layer': 26,
   'units_latent_layer': 6},
  'weighte

In [5]:
def instantiate_classifiers():
    classifiers = {
            "RF24": RandomForestClassifier(random_state=0, min_samples_leaf=2, min_samples_split=4),
            "RF4": RandomForestClassifier(random_state=0, max_depth=4),
            "RF5": RandomForestClassifier(random_state=0, max_depth=5),
            "RF7": RandomForestClassifier(random_state=0, max_depth=7)
        } 
    return classifiers

In [6]:
result_dict = defaultdict(list)

# for dataset_name in list(datasets.keys())[5:6]:
for dataset_name in list(datasets.keys()):
    print(dataset_name)
    result_dict["dataset"].append(dataset_name)
    X, y = datasets[dataset_name]['data'], datasets[dataset_name]['target']
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    n_splits = skf.get_n_splits(X, y)
    fold_result_dict = defaultdict(list)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        weights = ex.weights_calculation_strategy1(X_train, y_train)
        nn_config =  config[dataset_name]['nn_config']
        # pretrain ANN triplets
        embeddings_train, embeddings_test = ex.train_triplets(X_train, y_train, X_test, y_test, weights,nn_config)
        X_train_new = np.hstack([X_train, embeddings_train])
        X_test_new = np.hstack([X_test, embeddings_test])
        
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train, y_train)
            preds = clf.predict(X_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train_new, y_train)
            preds = clf.predict(X_test_new)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old+new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old+new-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(embeddings_train, y_train)
            preds = clf.predict(embeddings_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-new-features"].append(gmean)
    for k in fold_result_dict:
        result_dict[k].append(np.mean(fold_result_dict[k]))
        
            
            #f1, gmean
    # mean over folds
    # wiersze: datasety, 
    # kolumny -> klasyfikatory (przed dodaniem cech, po dodaniu, na samych nowych cechach)

1czysty-cut
Train Epoch: 1 [0/960 (0%)]	Loss: 2014.122314
Train Epoch: 1 [60/960 (67%)]	Loss: 420.958221

Test set: Average loss: 1443.2057

Train Epoch: 2 [0/960 (0%)]	Loss: 153.890457
Train Epoch: 2 [60/960 (67%)]	Loss: 72.200935

Test set: Average loss: 621.1189

Train Epoch: 3 [0/960 (0%)]	Loss: 36.700287
Train Epoch: 3 [60/960 (67%)]	Loss: 87.031433

Test set: Average loss: 371.3730

Train Epoch: 4 [0/960 (0%)]	Loss: 8.434065
Train Epoch: 4 [60/960 (67%)]	Loss: 47.292297

Test set: Average loss: 283.3458

Train Epoch: 5 [0/960 (0%)]	Loss: 3.580066
Train Epoch: 5 [60/960 (67%)]	Loss: 20.046930

Test set: Average loss: 211.7285

Train Epoch: 6 [0/960 (0%)]	Loss: 16.916737
Train Epoch: 6 [60/960 (67%)]	Loss: 22.806641

Test set: Average loss: 181.1538

Train Epoch: 7 [0/960 (0%)]	Loss: 19.036837
Train Epoch: 7 [60/960 (67%)]	Loss: 22.948460

Test set: Average loss: 167.3263

Train Epoch: 8 [0/960 (0%)]	Loss: 34.757587
Train Epoch: 8 [60/960 (67%)]	Loss: 15.289757

Test set: Average l

 25%|██▌       | 240/960 [00:00<00:00, 2397.12it/s]


Test set: Average loss: 58.5118



100%|██████████| 240/240 [00:00<00:00, 2913.95it/s]


Train Epoch: 1 [0/960 (0%)]	Loss: 3152.167969
Train Epoch: 1 [60/960 (67%)]	Loss: 665.049927

Test set: Average loss: 1486.4262

Train Epoch: 2 [0/960 (0%)]	Loss: 262.256592
Train Epoch: 2 [60/960 (67%)]	Loss: 78.535629

Test set: Average loss: 493.9693

Train Epoch: 3 [0/960 (0%)]	Loss: 86.875237
Train Epoch: 3 [60/960 (67%)]	Loss: 48.347008

Test set: Average loss: 346.5319

Train Epoch: 4 [0/960 (0%)]	Loss: 62.064388
Train Epoch: 4 [60/960 (67%)]	Loss: 27.560112

Test set: Average loss: 249.0561

Train Epoch: 5 [0/960 (0%)]	Loss: 78.746170
Train Epoch: 5 [60/960 (67%)]	Loss: 25.714626

Test set: Average loss: 176.3002

Train Epoch: 6 [0/960 (0%)]	Loss: 40.068207
Train Epoch: 6 [60/960 (67%)]	Loss: 14.563116

Test set: Average loss: 150.3674

Train Epoch: 7 [0/960 (0%)]	Loss: 15.925915
Train Epoch: 7 [60/960 (67%)]	Loss: 13.805833

Test set: Average loss: 129.5158

Train Epoch: 8 [0/960 (0%)]	Loss: 28.087721
Train Epoch: 8 [60/960 (67%)]	Loss: 13.239416

Test set: Average loss: 121.1

...

100%|██████████| 43/43 [00:00<00:00, 3254.04it/s]


Test set: Average loss: 0.3788



Train Epoch: 1 [0/172 (0%)]	Loss: 0.853558

Test set: Average loss: 0.8276

Train Epoch: 2 [0/172 (0%)]	Loss: 0.170191

Test set: Average loss: 1.0033

Train Epoch: 3 [0/172 (0%)]	Loss: 0.165244

Test set: Average loss: 0.6583

Train Epoch: 4 [0/172 (0%)]	Loss: 0.474318

Test set: Average loss: 0.7552

Train Epoch: 5 [0/172 (0%)]	Loss: 0.257905

Test set: Average loss: 0.4230

Train Epoch: 6 [0/172 (0%)]	Loss: 0.159808

Test set: Average loss: 0.4188

Train Epoch: 7 [0/172 (0%)]	Loss: 0.183233

Test set: Average loss: 0.3537

Train Epoch: 8 [0/172 (0%)]	Loss: 0.022147

Test set: Average loss: 0.3997

Train Epoch: 9 [0/172 (0%)]	Loss: 0.098536

Test set: Average loss: 0.4306

Train Epoch: 10 [0/172 (0%)]	Loss: 0.077536

Test set: Average loss: 0.5408

Train Epoch: 11 [0/172 (0%)]	Loss: 0.004889

Test set: Average loss: 0.6735

Train Epoch: 12 [0/172 (0%)]	Loss: 0.021619

Test set: Average loss: 0.5088

Train Epoch: 13 [0/172 (0%)]	Loss: 0.151455

Test set: Average loss: 0.6876

Train Ep

100%|██████████| 43/43 [00:00<00:00, 3338.80it/s]


Test set: Average loss: 0.5014



In [7]:
results_df = pd.DataFrame.from_dict(result_dict)

In [8]:
results_df

,dataset,RF24-f1-old-features,RF24-gmean-old-features,RF4-f1-old-features,RF4-gmean-old-features,RF5-f1-old-features,RF5-gmean-old-features,RF7-f1-old-features,RF7-gmean-old-features,RF24-f1-old+new-features,...,RF7-f1-old+new-features,RF7-gmean-old+new-features,RF24-f1-new-features,RF24-gmean-new-features,RF4-f1-new-features,RF4-gmean-new-features,RF5-f1-new-features,RF5-gmean-new-features,RF7-f1-new-features,RF7-gmean-new-features
0,1czysty-cut,0.927800,0.918606,0.905429,0.896495,0.905533,0.902969,0.928631,0.924891,0.949593,...,0.940664,0.919577,0.935059,0.917158,0.843592,0.799316,0.894887,0.867691,0.924217,0.903367
1,2delikatne-cut,0.785844,0.751689,0.760193,0.696414,0.777951,0.725794,0.786406,0.747480,0.769307,...,0.759618,0.706943,0.738049,0.692113,0.634186,0.442166,0.691096,0.599684,0.714505,0.652242
2,3mocniej-cut,0.573803,0.463827,0.498805,0.082434,0.578375,0.416180,0.584513,0.452692,0.577298,...,0.580165,0.441673,0.564296,0.431100,0.477765,0.077980,0.512662,0.286993,0.542016,0.375177
3,4delikatne-bezover-cut,0.818686,0.804856,0.829701,0.794818,0.827701,0.815232,0.833611,0.828137,0.825886,...,0.830549,0.802512,0.827759,0.800144,0.774515,0.703107,0.814804,0.765920,0.830106,0.794873
4,balance-scale,0.613201,0.096373,0.602859,0.096128,0.609438,0.096721,0.608746,0.095293,0.827131,...,0.836386,0.824691,0.825842,0.821758,0.823674,0.828008,0.810269,0.801573,0.826240,0.829062
5,cleveland,0.316170,0.083242,0.285711,0.073227,0.284363,0.074887,0.296950,0.080578,0.320964,...,0.341529,0.102859,0.275228,0.064066,0.302226,0.079639,0.306531,0.081135,0.270342,0.053455
6,cleveland_v2,0.296382,0.041099,0.269112,0.025991,0.266816,0.025892,0.323781,0.053368,0.351316,...,0.320213,0.063070,0.314682,0.055640,0.300404,0.049802,0.299471,0.046059,0.376748,0.074476
7,cmc,0.522486,0.507424,0.509892,0.479466,0.514592,0.487395,0.541080,0.523951,0.521219,...,0.537480,0.518117,0.483108,0.464024,0.490909,0.475536,0.497226,0.482161,0.505373,0.490035
8,dermatology,0.963844,0.955977,0.966801,0.958125,0.969889,0.961607,0.969889,0.961607,0.939425,...,0.942697,0.934326,0.924104,0.911319,0.933114,0.921721,0.930500,0.920093,0.920908,0.908305
9,glass,0.704754,0.494085,0.638555,0.294955,0.688237,0.419342,0.704035,0.489533,0.629026,...,0.605948,0.255736,0.502937,0.155359,0.446911,0.094853,0.479883,0.125597,0.525060,0.164806


In [9]:
results_df.filter(regex=("dataset|RF24-f1*"))

,dataset,RF24-f1-old-features,RF24-f1-old+new-features,RF24-f1-new-features
0,1czysty-cut,0.927800,0.949593,0.935059
1,2delikatne-cut,0.785844,0.769307,0.738049
2,3mocniej-cut,0.573803,0.577298,0.564296
3,4delikatne-bezover-cut,0.818686,0.825886,0.827759
4,balance-scale,0.613201,0.827131,0.825842
5,cleveland,0.316170,0.320964,0.275228
6,cleveland_v2,0.296382,0.351316,0.314682
7,cmc,0.522486,0.521219,0.483108
8,dermatology,0.963844,0.939425,0.924104
9,glass,0.704754,0.629026,0.502937


In [10]:
results_df.filter(regex=("dataset|RF24-gmean*"))

,dataset,RF24-gmean-old-features,RF24-gmean-old+new-features,RF24-gmean-new-features
0,1czysty-cut,0.918606,0.930663,0.917158
1,2delikatne-cut,0.751689,0.728470,0.692113
2,3mocniej-cut,0.463827,0.456498,0.431100
3,4delikatne-bezover-cut,0.804856,0.803852,0.800144
4,balance-scale,0.096373,0.814358,0.821758
5,cleveland,0.083242,0.071429,0.064066
6,cleveland_v2,0.041099,0.079709,0.055640
7,cmc,0.507424,0.505827,0.464024
8,dermatology,0.955977,0.930146,0.911319
9,glass,0.494085,0.335981,0.155359


In [11]:
results_df.filter(regex=("dataset|RF4-f1*"))

,dataset,RF4-f1-old-features,RF4-f1-old+new-features,RF4-f1-new-features
0,1czysty-cut,0.905429,0.899926,0.843592
1,2delikatne-cut,0.760193,0.715853,0.634186
2,3mocniej-cut,0.498805,0.500066,0.477765
3,4delikatne-bezover-cut,0.829701,0.813597,0.774515
4,balance-scale,0.602859,0.816956,0.823674
5,cleveland,0.285711,0.309718,0.302226
6,cleveland_v2,0.269112,0.304782,0.300404
7,cmc,0.509892,0.522542,0.490909
8,dermatology,0.966801,0.939453,0.933114
9,glass,0.638555,0.570479,0.446911


In [12]:
results_df.filter(regex=("dataset|RF4-gmean*"))

,dataset,RF4-gmean-old-features,RF4-gmean-old+new-features,RF4-gmean-new-features
0,1czysty-cut,0.896495,0.864862,0.799316
1,2delikatne-cut,0.696414,0.625296,0.442166
2,3mocniej-cut,0.082434,0.193539,0.077980
3,4delikatne-bezover-cut,0.794818,0.771603,0.703107
4,balance-scale,0.096128,0.824959,0.828008
5,cleveland,0.073227,0.072200,0.079639
6,cleveland_v2,0.025991,0.051850,0.049802
7,cmc,0.479466,0.501457,0.475536
8,dermatology,0.958125,0.930139,0.921721
9,glass,0.294955,0.173365,0.094853


In [13]:
results_df.filter(regex=("dataset|RF5-f1*"))

,dataset,RF5-f1-old-features,RF5-f1-old+new-features,RF5-f1-new-features
0,1czysty-cut,0.905533,0.935141,0.894887
1,2delikatne-cut,0.777951,0.766273,0.691096
2,3mocniej-cut,0.578375,0.543750,0.512662
3,4delikatne-bezover-cut,0.827701,0.838766,0.814804
4,balance-scale,0.609438,0.824386,0.810269
5,cleveland,0.284363,0.311731,0.306531
6,cleveland_v2,0.266816,0.332075,0.299471
7,cmc,0.514592,0.519577,0.497226
8,dermatology,0.969889,0.939453,0.930500
9,glass,0.688237,0.597556,0.479883


In [14]:
results_df.filter(regex=("dataset|RF5-gmean*"))

,dataset,RF5-gmean-old-features,RF5-gmean-old+new-features,RF5-gmean-new-features
0,1czysty-cut,0.902969,0.906888,0.867691
1,2delikatne-cut,0.725794,0.699574,0.599684
2,3mocniej-cut,0.416180,0.352246,0.286993
3,4delikatne-bezover-cut,0.815232,0.805011,0.765920
4,balance-scale,0.096721,0.819558,0.801573
5,cleveland,0.074887,0.081826,0.081135
6,cleveland_v2,0.025892,0.072633,0.046059
7,cmc,0.487395,0.498159,0.482161
8,dermatology,0.961607,0.930139,0.920093
9,glass,0.419342,0.186148,0.125597


In [15]:
results_df.filter(regex=("dataset|RF7-f1*"))

,dataset,RF7-f1-old-features,RF7-f1-old+new-features,RF7-f1-new-features
0,1czysty-cut,0.928631,0.940664,0.924217
1,2delikatne-cut,0.786406,0.759618,0.714505
2,3mocniej-cut,0.584513,0.580165,0.542016
3,4delikatne-bezover-cut,0.833611,0.830549,0.830106
4,balance-scale,0.608746,0.836386,0.826240
5,cleveland,0.296950,0.341529,0.270342
6,cleveland_v2,0.323781,0.320213,0.376748
7,cmc,0.541080,0.537480,0.505373
8,dermatology,0.969889,0.942697,0.920908
9,glass,0.704035,0.605948,0.525060


In [16]:
results_df.filter(regex=("dataset|RF7-gmean*"))

,dataset,RF7-gmean-old-features,RF7-gmean-old+new-features,RF7-gmean-new-features
0,1czysty-cut,0.924891,0.919577,0.903367
1,2delikatne-cut,0.747480,0.706943,0.652242
2,3mocniej-cut,0.452692,0.441673,0.375177
3,4delikatne-bezover-cut,0.828137,0.802512,0.794873
4,balance-scale,0.095293,0.824691,0.829062
5,cleveland,0.080578,0.102859,0.053455
6,cleveland_v2,0.053368,0.063070,0.074476
7,cmc,0.523951,0.518117,0.490035
8,dermatology,0.961607,0.934326,0.908305
9,glass,0.489533,0.255736,0.164806
